In [1]:
import os
import sys

notebook_dir = os.path.dirname(os.path.abspath('__file__'))
os.chdir(notebook_dir)

sys.path.append(os.path.abspath('../src'))

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset

# import from src/
from data import DataPreparation
from models import TransformerModel

In [3]:
# Load the technical indicators data
file_path = '../data/processed/market_features.csv'
data_prep = DataPreparation(file_path)
X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor = data_prep.prepare_data()


In [16]:
# Hyperparameters
input_dim = X_train_tensor.shape[1]
num_classes = len(y_train_tensor.unique())  

d_model = 128
num_layers = 3  
dim_feedforward = 512  
nhead = 4
b_size = 32
lr = 9.930716357727735e-05

In [17]:
# Check for GPU

# ON CUDA
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ON MAC
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
print("Device Used", device)

# Initialize model, loss function, and optimizer
model = TransformerModel(input_dim, num_classes, d_model=d_model, nhead=nhead, num_layers=num_layers, dim_feedforward=dim_feedforward).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# Prepare DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=b_size, shuffle=True)

val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=b_size, shuffle=False)

# Training loop with plotting and early stopping
epochs = 200
train_losses = []
val_losses = []
best_val_loss = float('inf')
patience = 5
trigger_times = 0

Device Used mps


In [18]:
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    epoch_train_loss = running_loss / len(train_loader)
    train_losses.append(epoch_train_loss)

    # Validation phase
    model.eval()
    val_running_loss = 0.0
    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
            val_outputs = model(val_inputs)
            val_loss = criterion(val_outputs, val_labels)
            val_running_loss += val_loss.item()
    epoch_val_loss = val_running_loss / len(val_loader)
    val_losses.append(epoch_val_loss)

    print(f'Epoch {epoch+1}/{epochs}, Training Loss: {epoch_train_loss}, Validation Loss: {epoch_val_loss}')

    # Early stopping
    if epoch_val_loss < best_val_loss:
        best_val_loss = epoch_val_loss
        trigger_times = 0
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print(f'Early stopping at epoch {epoch+1}')
            break

    # Scheduler for learning rate
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2)
    scheduler.step(epoch_val_loss)

KeyboardInterrupt: 

In [ ]:
# Plotting training and validation losses
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss over Epochs')
plt.show()